In [1]:

import argparse
from argparse import RawDescriptionHelpFormatter
import yaml
from yaml.loader import SafeLoader
import logging
import os
import shutil
import sys
import pandas as pd
import json
import numpy as np


logger = logging.getLogger("General")


In [2]:
def read_json(json_path):
    with open(json_path) as json_data:
        d = json.loads(json_data)
    return d

def read_yaml(path_to_config_file):
    """
    Input:
    the path to the config file

    Returns
    a dictionary that contains all the config info

    """
    with open(path_to_config_file, "r") as v:
        config = yaml.load(v, Loader=SafeLoader)

    return config

In [13]:
l1_glm_yaml=read_yaml('/media/tlei/data/soft/MRIworkflow/example_config.yaml')

In [16]:
l1_glm_yaml['general']["print_cmd_only"]==False

False

In [35]:
# Define the basic_contrasts dictionary
basic_contrasts = {
    "adult": np.array([1, 0, 0, 0, 0, 0, 0, 0, 0]),
    "child": np.array([0, 1, 0, 0, 0, 0, 0, 0, 0]),
    "body": np.array([0, 0, 1, 0, 0, 0, 0, 0, 0]),
    "limb": np.array([0, 0, 0, 1, 0, 0, 0, 0, 0]),
    "JP_word": np.array([0, 0, 0, 0, 1, 0, 0, 0, 0]),
    "JP_FF": np.array([0, 0, 0, 0, 0, 1, 0, 0, 0]),
    "JP_CB": np.array([0, 0, 0, 0, 0, 0, 1, 0, 0]),
    "JP_CS": np.array([0, 0, 0, 0, 0, 0, 0, 1, 0]),
    "JP_SC": np.array([0, 0, 0, 0, 0, 0, 0, 0, 1])
}
contrasts_orig = {
        "AllvsNull": (
            basic_contrasts["adult"] 
            + basic_contrasts["child"] 
            + basic_contrasts["body"] 
            + basic_contrasts["limb"] 
            + basic_contrasts["JP_word"] 
            + basic_contrasts["JP_FF"] 
            + basic_contrasts["JP_CB"] 
            + basic_contrasts["JP_CS"] 
            + basic_contrasts["JP_SC"] 
        ),
        "PERvsNull": (
            basic_contrasts["JP_CB"] 
            + basic_contrasts["JP_SC"] 
        ),
        "LEXvsNull": (
            basic_contrasts["JP_CS"] 
            + basic_contrasts["JP_FF"] 
        ),    
        "PERvsLEX": (
            basic_contrasts["JP_CB"] / 2
            + basic_contrasts["JP_SC"] / 2
            - basic_contrasts["JP_CS"] / 2
            - basic_contrasts["JP_FF"] / 2
        ),          
        "WordvsLEX": (
            basic_contrasts["JP_word"] 
            - basic_contrasts["JP_CS"] / 2
            - basic_contrasts["JP_FF"] / 2
        ),
        }

In [36]:
contrasts_orig

{'AllvsNull': array([1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'PERvsNull': array([0, 0, 0, 0, 0, 0, 1, 0, 1]),
 'LEXvsNull': array([0, 0, 0, 0, 0, 1, 0, 1, 0]),
 'PERvsLEX': array([ 0. ,  0. ,  0. ,  0. ,  0. , -0.5,  0.5, -0.5,  0.5]),
 'WordvsLEX': array([ 0. ,  0. ,  0. ,  0. ,  1. , -0.5,  0. , -0.5,  0. ])}

In [73]:
l1_glm_yaml['contrasts']

['AllvsNull',
 'PERvsNull',
 'LEXvsNull',
 'PERvsLEX',
 'WordvsLEX',
 'WordvsPER',
 'WordvsLEXPER',
 'WordvsAllnoWordnoLEX',
 'WordvsAllnoWord',
 'LEXvsAllnoWordnoLEX',
 'PERvsAllnoWordnoLEX',
 'SCvsCB',
 'CSvsFF',
 'FacesvsNull',
 'FacesvsLEX',
 'FacesvsPER',
 'FacesvsLEXPER',
 'FacesvsAllnoFace',
 'AdultvsChild',
 'LimbsvsNull',
 'LimbsvsLEX',
 'LimbsvsPER',
 'LimbsvsLEXPER',
 'LimbsvsAllnoLimbs',
 'BodysLimb']

In [74]:
l1_glm_yaml['contrasts_group']

{'All': ['adult',
  'child',
  'body',
  'limb',
  'JP_word',
  'JP_FF',
  'JP_CB',
  'JP_CS',
  'JP_SC'],
 'AllnoWord': ['adult',
  'child',
  'body',
  'limb',
  'JP_FF',
  'JP_CB',
  'JP_CS',
  'JP_SC'],
 'AllnoWordnoLEX': ['adult',
  'child',
  'body',
  'limb',
  'JP_FF',
  'JP_CB',
  'JP_CS',
  'JP_SC'],
 'LEXPER': ['JP_FF', 'JP_CB', 'JP_CS', 'JP_SC'],
 'PER': ['JP_CB', 'JP_SC'],
 'LEX': ['JP_CS', 'JP_FF'],
 'Faces': ['adult', 'child'],
 'Word': ['JP_word'],
 'Limbs': ['body', 'limb']}

In [152]:
# Extract contrasts
contrasts_yaml = l1_glm_yaml['contrasts']
contrast_groups=l1_glm_yaml['contrast_groups']
contrasts = {}
for contrast in contrasts_yaml:
    contrast_vector = np.zeros_like(next(iter(basic_contrasts.values())), dtype=np.float64)
    plus=contrast.split('vs')[0]
    minus=contrast.split('vs')[1]
    try:
        if minus == "Null":
            for part in contrast_groups[plus]:
                contrast_vector += basic_contrasts[part]
            contrasts[contrast]=contrast_vector
        else:
            con1=contrast_groups[plus]
            factor1=len(con1)
            con2=contrast_groups[minus]
            factor2=len(con2)
            for part in con1:
                contrast_vector += basic_contrasts[part]/factor1
            for part in con2:
                contrast_vector -= basic_contrasts[part]/factor2        
            contrasts[contrast]=contrast_vector
    except KeyError as e :
        logger.error(f"{contrast} contrast have key error, check your yaml")



In [118]:
contrasts

{'AllvsNull': array([1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'PERvsNull': array([0., 0., 0., 0., 0., 0., 1., 0., 1.]),
 'LEXvsNull': array([0., 0., 0., 0., 0., 1., 0., 1., 0.]),
 'PERvsLEX': array([ 0. ,  0. ,  0. ,  0. ,  0. , -0.5,  0.5, -0.5,  0.5]),
 'WordvsLEX': array([ 0. ,  0. ,  0. ,  0. ,  1. , -0.5,  0. , -0.5,  0. ]),
 'WordvsPER': array([ 0. ,  0. ,  0. ,  0. ,  1. ,  0. , -0.5,  0. , -0.5]),
 'WordvsLEXPER': array([ 0.  ,  0.  ,  0.  ,  0.  ,  1.  , -0.25, -0.25, -0.25, -0.25]),
 'WordvsAllnoWordnoLEX': array([-0.125, -0.125, -0.125, -0.125,  1.   , -0.125, -0.125, -0.125,
        -0.125]),
 'WordvsAllnoWord': array([-0.125, -0.125, -0.125, -0.125,  1.   , -0.125, -0.125, -0.125,
        -0.125]),
 'LEXvsAllnoWordnoLEX': array([-0.125, -0.125, -0.125, -0.125,  0.   ,  0.375, -0.125,  0.375,
        -0.125]),
 'PERvsAllnoWordnoLEX': array([-0.125, -0.125, -0.125, -0.125,  0.   , -0.125,  0.375, -0.125,
         0.375]),
 'SCvsCB': array([ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0

In [160]:
contrasts_orig = {
        "AllvsNull": (
            basic_contrasts["adult"] 
            + basic_contrasts["child"] 
            + basic_contrasts["body"] 
            + basic_contrasts["limb"] 
            + basic_contrasts["JP_word"] 
            + basic_contrasts["JP_FF"] 
            + basic_contrasts["JP_CB"] 
            + basic_contrasts["JP_CS"] 
            + basic_contrasts["JP_SC"] 
        ),
        "PERvsNull": (
            basic_contrasts["JP_CB"] 
            + basic_contrasts["JP_SC"] 
        ),
        "LEXvsNull": (
            basic_contrasts["JP_CS"] 
            + basic_contrasts["JP_FF"] 
        ),    
        "PERvsLEX": (
            basic_contrasts["JP_CB"] / 2
            + basic_contrasts["JP_SC"] / 2
            - basic_contrasts["JP_CS"] / 2
            - basic_contrasts["JP_FF"] / 2
        ),          
        "WordvsLEX": (
            basic_contrasts["JP_word"] 
            - basic_contrasts["JP_CS"] / 2
            - basic_contrasts["JP_FF"] / 2
        ),  
        "WordvsPER": (
            basic_contrasts["JP_word"] 
            - basic_contrasts["JP_CB"] / 2
            - basic_contrasts["JP_SC"] / 2
        ),   
        "WordvsLEXPER": (
            basic_contrasts["JP_word"] 
            - basic_contrasts["JP_CS"] / 4
            - basic_contrasts["JP_FF"] / 4
            - basic_contrasts["JP_CB"] / 4
            - basic_contrasts["JP_SC"] / 4
        ),     
        "WordvsAllnoWordnoLEX": (
            basic_contrasts["JP_word"] 
            - basic_contrasts["JP_CB"] / 6
            - basic_contrasts["JP_SC"] / 6
            - basic_contrasts["body"] / 6
            - basic_contrasts["limb"] / 6
            - basic_contrasts["adult"] / 6
            - basic_contrasts["child"] / 6
        ),
        
        "WordvsAllnoWord": (
            basic_contrasts["JP_word"] 
            - basic_contrasts["JP_CS"] / 8
            - basic_contrasts["JP_FF"] / 8                
            - basic_contrasts["JP_CB"] / 8
            - basic_contrasts["JP_SC"] / 8
            - basic_contrasts["body"] / 8
            - basic_contrasts["limb"] / 8
            - basic_contrasts["adult"] / 8
            - basic_contrasts["child"] / 8
        ),     
        "LEXvsAllnoWordnoLEX": (
            basic_contrasts["JP_CS"] / 2
            + basic_contrasts["JP_FF"] / 2
            - basic_contrasts["JP_CB"] / 6
            - basic_contrasts["JP_SC"] / 6
            - basic_contrasts["body"] / 6
            - basic_contrasts["limb"] / 6
            - basic_contrasts["adult"] / 6
            - basic_contrasts["child"] / 6
        ),     

        "SCvsCB": (
            basic_contrasts["JP_SC"] 
            - basic_contrasts["JP_CB"] 
            
        ),     
        "CSvsFF": (
            basic_contrasts["JP_CS"] 
            - basic_contrasts["JP_FF"] 
            
        ),     
        "FacesvsNull": (
            basic_contrasts["adult"] 
            + basic_contrasts["child"] 
        ),    
        "FacesvsLEX": (
            basic_contrasts["adult"] / 2
            + basic_contrasts["child"] / 2
            - basic_contrasts["JP_CS"] / 2
            - basic_contrasts["JP_FF"]  / 2
        ), 
        "FacesvsPER": (
            basic_contrasts["adult"] / 2
            + basic_contrasts["child"] / 2
            - basic_contrasts["JP_CB"] / 2
            - basic_contrasts["JP_SC"]  / 2
        ),    
        "FacesvsLEXPER": (
            basic_contrasts["adult"] / 2
            + basic_contrasts["child"] / 2
            - basic_contrasts["JP_CB"] / 4
            - basic_contrasts["JP_SC"]  / 4
            - basic_contrasts["JP_CS"] / 4
            - basic_contrasts["JP_FF"]  / 4                
        ),   
        "FacesvsAllnoFace": (
            basic_contrasts["adult"] / 2
            + basic_contrasts["child"] / 2
            - basic_contrasts["JP_CB"] / 7
            - basic_contrasts["JP_SC"]  / 7
            - basic_contrasts["JP_CS"] / 7
            - basic_contrasts["JP_FF"]  / 7  
            - basic_contrasts["body"] / 7
            - basic_contrasts["limb"] / 7 
            - basic_contrasts["JP_word"] / 7            
        ),   
        "AdultvsChild": (
            basic_contrasts["adult"] 
            - basic_contrasts["child"] 
        ),        
        
        "LimbsvsNull": (
            basic_contrasts["body"] 
            + basic_contrasts["limb"] 
        ),    
        "LimbsvsLEX": (
            basic_contrasts["body"] / 2
            + basic_contrasts["limb"] / 2
            - basic_contrasts["JP_CS"] / 2
            - basic_contrasts["JP_FF"]  / 2
        ), 
        "LimbsvsPER": (
            basic_contrasts["body"] / 2
            + basic_contrasts["limb"] / 2
            - basic_contrasts["JP_CB"] / 2
            - basic_contrasts["JP_SC"]  / 2
        ),    
        "LimbsvsLEXPER": (
            basic_contrasts["body"] / 2
            + basic_contrasts["limb"] / 2
            - basic_contrasts["JP_CB"] / 4
            - basic_contrasts["JP_SC"]  / 4
            - basic_contrasts["JP_CS"] / 4
            - basic_contrasts["JP_FF"]  / 4                
        ),   
        "LimbsvsAllnoLimbs": (
            basic_contrasts["body"] / 2
            + basic_contrasts["limb"] / 2
            - basic_contrasts["JP_CB"] / 7
            - basic_contrasts["JP_SC"]  / 7
            - basic_contrasts["JP_CS"] / 7
            - basic_contrasts["JP_FF"]  / 7  
            - basic_contrasts["adult"] / 7
            - basic_contrasts["child"] / 7 
            - basic_contrasts["JP_word"] / 7              
        ), 
        "BodyvsLimb": (
            basic_contrasts["body"] 
            - basic_contrasts["limb"] 
        )                                                                                                                                                        
        
    }

In [126]:
def compare_dicts_with_tolerance(dict1, dict2, tol=1e-8):
    if dict1.keys() != dict2.keys():
        return False
    
    for key in dict1:
        if not np.allclose(dict1[key], dict2[key], atol=tol):
            print(f"Mismatch found at key: {key}")
            return False
    
    return True


In [161]:
compare_dicts_with_tolerance(contrasts,contrasts_orig)

True

In [156]:
contrasts_orig['FacesvsAllnoFace']

array([ 0.5       ,  0.5       , -0.16666667, -0.16666667,  0.        ,
       -0.16666667, -0.16666667, -0.16666667, -0.16666667])

In [157]:
contrasts['FacesvsAllnoFace']

array([ 0.5       ,  0.5       , -0.14285714, -0.14285714, -0.14285714,
       -0.14285714, -0.14285714, -0.14285714, -0.14285714])